In [16]:
# !pip install crewai crewai_tools langchain_community

In [17]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [18]:
from crewai import Agent, Task, Crew, LLM

In [19]:
choose_LLM = 1
colab_exp = 0
import os

from langchain_openai import ChatOpenAI

if choose_LLM==1:
    if(colab_exp):
        from google.colab import userdata    ###for colab use
        llm = ChatOpenAI(model = "gpt-4o-mini",openai_api_key=userdata.get('OPENAI_API_KEY'))
    else:
        from dotenv import load_dotenv
        load_dotenv() 
        llm = "gpt-4o-mini" #ChatOpenAI(model = "gpt-4o-mini",openai_api_key=os.getenv("OPENAI_API_KEY"))

else:
    llm = ChatOpenAI(model="ollama/llama3.2", base_url="http://localhost:11434",temperature=0.1)
    


In [20]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

# from google.colab import userdata    ###for colab use
# serper_key = userdata.get('SERPER_key')

serper_key = os.getenv("SERPER_API_KEY")
os.environ["SERPER_API_KEY"] = serper_key

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

In [21]:
data_analyst_agent = Agent(
    role="Data Analyst",
    goal="Monitor and analyze market data in real-time "
         "to identify trends and predict market movements.",
    backstory="Specializing in financial markets, this agent "
              "uses statistical modeling and machine learning "
              "to provide crucial insights. With a knack for data, "
              "the Data Analyst Agent is the cornerstone for "
              "informing trading decisions.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [22]:
trading_strategy_agent = Agent(
    role="Trading Strategy Developer",
    goal="Develop and test various trading strategies based "
         "on insights from the Data Analyst Agent.",
    backstory="Equipped with a deep understanding of financial "
              "markets and quantitative analysis, this agent "
              "devises and refines trading strategies. It evaluates "
              "the performance of different approaches to determine "
              "the most profitable and risk-averse options.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [23]:
# Task for Data Analyst Agent: Analyze Market Data
data_analysis_task = Task(
    description=(
        "Continuously monitor and analyze market data for "
        "the selected stock ({stock_selection}). "
        "Use statistical modeling and machine learning to "
        "identify trends and predict market movements."
    ),
    expected_output=(
        "Insights and alerts about significant market "
        "opportunities or threats for {stock_selection}."
    ),
    agent=data_analyst_agent,
)

In [24]:
# Task for Trading Strategy Agent: Develop Trading Strategies
strategy_development_task = Task(
    description=(
        "Develop and refine trading strategies based on "
        "the insights from the Data Analyst and "
        "user-defined risk tolerance ({risk_tolerance}). "
        "Consider trading preferences ({trading_strategy_preference})."
    ),
    expected_output=(
        "A set of potential trading strategies for {stock_selection} "
        "that align with the user's risk tolerance."
    ),
    agent=trading_strategy_agent,
)

In [25]:
from crewai import Crew, Process
from langchain_openai import ChatOpenAI

# Define the crew with agents and tasks
financial_trading_crew = Crew(
    agents=[data_analyst_agent, 
            trading_strategy_agent ],
    
    tasks=[data_analysis_task, 
           strategy_development_task ],
    
    manager_llm=llm,
    process=Process.hierarchical,
    verbose=True
)

In [26]:
# Example data for kicking off the process
financial_trading_inputs = {
    'stock_selection': 'NVDA',
    'risk_tolerance': 'Medium',
    'trading_strategy_preference': 'Day Trading',
}

In [27]:
### this execution will take some time to run
result = financial_trading_crew.kickoff(inputs=financial_trading_inputs)

2025-10-31 22:23:33,856 - 8783415296 - llm.py-llm:170 - ERROR: LiteLLM call failed: litellm.RateLimitError: RateLimitError: OpenAIException - Error code: 429 - {'error': {'message': 'Request too large for gpt-4o-mini in organization org-rZOR5A4vLgEAZGYa0OLQQdl3 on tokens per min (TPM): Limit 200000, Requested 403945. The input or output tokens must be reduced in order to run successfully. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}


2025-10-31 22:26:20,157 - 8783415296 - llm.py-llm:170 - ERROR: LiteLLM call failed: litellm.RateLimitError: RateLimitError: OpenAIException - Error code: 429 - {'error': {'message': 'Request too large for gpt-4o-mini in organization org-rZOR5A4vLgEAZGYa0OLQQdl3 on tokens per min (TPM): Limit 200000, Requested 403516. The input or output tokens must be reduced in order to run successfully. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}


2025-10-31 22:26:46,437 - 8783415296 - llm.py-llm:170 - ERROR: LiteLLM call failed: litellm.RateLimitError: RateLimitError: OpenAIException - Error code: 429 - {'error': {'message': 'Request too large for gpt-4o-mini in organization org-rZOR5A4vLgEAZGYa0OLQQdl3 on tokens per min (TPM): Limit 200000, Requested 403787. The input or output tokens must be reduced in order to run successfully. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}


2025-10-31 22:27:13,565 - 8783415296 - llm.py-llm:170 - ERROR: LiteLLM call failed: litellm.RateLimitError: RateLimitError: OpenAIException - Error code: 429 - {'error': {'message': 'Request too large for gpt-4o-mini in organization org-rZOR5A4vLgEAZGYa0OLQQdl3 on tokens per min (TPM): Limit 200000, Requested 403739. The input or output tokens must be reduced in order to run successfully. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}




LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.

# Agent: Trading Strategy Developer
## Task: Can you please explain your approach to using ARIMA and GARCH models for the analysis of NVDA stock data? Specifically, what key indicators do you typically look for that might signal market movements, and how do you backtest your models to ensure their reliability?


# Agent: Trading Strategy Developer
## Thought: I need to gather information about using ARIMA and GARCH models for analyzing stock data, particularly for NVDA stock. This will help me provide a detailed response that aligns with the context shared.
## Using tool: Search the internet
## Tool Input: 
"{\"search_query\": \"ARIMA GARCH models analysis NVDA stock key indicators backtesting\"}"
## Tool Output: 

Search results: Title: [PDF] Predicting NVIDIA's Next-Day Stock Price: A Comparative Analysis ...
Link: https://arxiv.org/pdf/2405.08284?
Snippet: Our study analyzes four key models: ARIMA, LST

In [28]:
from IPython.display import display, Markdown
Markdown(result.raw)

**Potential Trading Strategies for NVDA Aligned with Medium Risk Tolerance:**

1. **Momentum Trading Strategy**:
   - **Use Case**: Designed for short-term trading (day trading) based on the momentum of stock price movements.
   - **Indicators**: 
     - Utilize ARIMA for time series forecasting to identify upward momentum in NVDA stock prices.
     - Implementation of buy signals when the price surpasses the predicted ARIMA price, and sell signals when it falls below the predicted level.
   - **Trading Rule**: Enter position when momentum indicators (like moving averages) signal an upward trend; exit position if the price reverses.

2. **Volatility Breakout Strategy**:
   - **Use Case**: This strategy exploits periods of high volatility determined through GARCH models.
   - **Indicators**: 
     - Analyze GARCH model outputs for predicted volatility. 
     - Execute buy orders when NVDA’s price penetrates upper volatility bounds, and sell orders when it breaches lower volatility limits.
   - **Trading Rule**: Enter position on breakout events; cut losses when the price closes back within the volatility bands established.

3. **Statistical Arbitrage Strategy**:
   - **Use Case**: Ideal for capturing short-lived price discrepancies through statistical techniques.
   - **Indicators**:
     - Continuously monitor price correlations and use ARIMA to predict mean-reversion of NVDA prices.
     - Establish a hedge by taking opposite positions in correlated assets (e.g., tech sector ETFs).
   - **Trading Rule**: Buy NVDA when price drops significantly compared to historical averages; sell once it returns to the predicted mean.

4. **Risk Managed Positioning**:
   - **Use Case**: Aimed at creating a flexible portfolio that adjusts based on GARCH-derived risk assessments.
   - **Indicators**:
     - Periodically review GARCH models to adjust the position size based on the predicted volatility – larger positions in low volatility periods and smaller in high volatility periods.
   - **Trading Rule**: Enter with a position size effectively moderated by anticipated risk levels to manage exposure while maximizing profit potential.

5. **Event-Based Trading Strategy**:
   - **Use Case**: Focus on taking advantage of corporate events (earnings reports, product launches).
   - **Indicators**:
     - Utilize historical data and ARIMA forecasts to anticipate stock price movements surrounding significant corporate announcements.
     - Prepare for potential volatility spikes as observed through GARCH forecasts.
   - **Trading Rule**: Buy NVDA shares in anticipation of positive earnings surprises, and consider protective stops to mitigate downside risks following the announcements.

Each of these strategies utilizes the insights derived from ARIMA and GARCH models to craft approaches tailored to a medium-risk tolerance level, specifically accommodating day trading preferences. The combination of predictive modeling and risk assessment ensures the strategies are well-rounded, aiming for both profit realization and loss limitation.